# Week 3 report

Haoyang Han

Based on Week 2's report, what we need to do is correctly get the label for training. Luckily we get real-world data for labeling and designed the metric with maximum accuracy. Here is the outline of Week 3's report(given by different jupyter notebook)

---

1. Define labeling data's timesheet(**get_time_epoch.ipynb**)

2. Label the gps_observation_points(**labeling data.ipynb**)

3. Fit the model again.(**fitting model.ipynb**)
    



In [1]:
# !pip install brewer2mpl
# import necessary package

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

pd.set_option('display.max_row',100)
pd.set_option('display.max_column',100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}


plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

# Version
print(mpl.__version__)  #> 3.0.0
print(sns.__version__)  #> 0.9.0

3.0.2
0.9.0


/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kevin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


###  import data and description

In [3]:
!ls

Define metric, feature engineering.ipynb
PySQL_script.py
True.csv
corr_timesheet.csv
get_time_epoch.ipynb
labeled_data.csv
labeling data.ipynb
local_time_attack.csv


In [4]:
data = pd.read_csv('labeled_data.csv')

In [9]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,7 days 3 hours 26 mins
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_kevin_nwbo45
H2O cluster total nodes:,1
H2O cluster free memory:,3.949 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [38]:
h2o.shutdown(prompt = True)

[WARNING] in <ipython-input-38-1ea49f0103fa> line 1:
    >>> h2o.shutdown(prompt = True)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
Are you sure you want to shutdown the H2O instance running at http://localhost:54321 (Y/N)? Y
H2O session _sid_a937 closed.


### Fitting the model

In [10]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [11]:
data1 = h2o.import_file('labeled_data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
cols = data1.col_names

In [25]:
cols

['C1',
 'id',
 'sysTime',
 'Lat',
 'Long',
 'Alt',
 'Provider',
 'GPSTime',
 'FixSatCount',
 'HasRadialAccuracy',
 'HasVerticalAccuracy',
 'ElapsedRealtimeNanos',
 'HasSpeed',
 'HasSpeedAccuracy',
 'Speed',
 'SpeedAccuracy',
 'HasBearing',
 'HasBearingAccuracy',
 'Bearing',
 'BearingAccuracy',
 'data_dump',
 'n',
 'Attack']

In [26]:
cols.remove('GPSTime') # avoid overfitting

In [28]:
#split the data as described above
train, valid, test = data1.split_frame([0.7, 0.15], seed=1234)

#Prepare predictors and response columns
X_train = cols[:-1]     #last column is Attack, our desired response variable 
y_train = cols[-1]    

In [29]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [34]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [35]:
rf_fit1.train(X_train, y_train, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [36]:
rf_perf1 = rf_fit1.model_performance(test)

In [44]:
# Retreive test set auc
print(rf_perf1.auc())
print(rf_perf1.confusion_matrix())

0.9999999985352123
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46845046937465673: 


,False,True,Error,Rate
False,36454.0,1.0,0.0,(1.0/36455.0)
True,0.0,18727.0,0.0,(0.0/18727.0)
Total,36454.0,18728.0,0.0,(1.0/55182.0)


In [38]:
# try more complicated model
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit1', ntrees = 200, seed=1)

In [39]:
rf_fit2.train(X_train, y_train, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
rf_perf2 = rf_fit2.model_performance(test)

In [45]:
# Retreive test set mse
print(rf_perf2.auc())
print(rf_perf2.confusion_matrix())

0.9999999978028185
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5267208081949502: 


,False,True,Error,Rate
False,36454.0,1.0,0.0,(1.0/36455.0)
True,0.0,18727.0,0.0,(0.0/18727.0)
Total,36454.0,18728.0,0.0,(1.0/55182.0)


We can see that this confusion matrix is really great. Now we need higher amount of local_time_attack to see whether model is accurate enough.